In [1]:
!pip install html5lib beautifulsoup4
!pip install selenium

from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
import pandas as pd

In [2]:
from urllib import request
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [3]:
from selenium.webdriver.common.keys import Keys

In [4]:
import numpy as np

In [36]:
# 인코딩이 euc-kr입니다. 저걸 지정하지 않으면 csv가 안 열려요!
# idol_list_withid를 불러오셔야 합니다. 멜론 url에 가수를 찾기 위한 id입니다. 

idol_list = pd.read_csv('idol_list_idfix.csv', encoding='euc-kr')
idol_list.head()

,artist,artist_eng,member_num,leader_name,gender,agency,idol_id
0,서태지와 아이들,SeoTaiji And Boys,3,서태지,남,요요 기획,100161.0
1,더 블루,The Blue,2,손지창,남,SM 엔터테인먼트,180144.0
2,잼,Zam,5,조진수,혼,DSP 미디어,225786.0
3,에스오에스,S.O.S,4,NaN,여,삼포니 레코드,101645.0
4,솔리드,Solid,3,정재윤,남,비앤비,100037.0


In [37]:
idol_list.shape

(338, 7)

In [38]:
# 멜론에 정보가 없어서 아예 페이지가 없었던 가수들이 몇몇 있습니다.
# 이런 경우 id가 공란이기에 수집이 불가합니다. 모두 드랍하겠습니다. 

idol_list = idol_list[np.isfinite(idol_list['idol_id'])].reset_index(drop=True)

In [39]:
idol_list.shape

(332, 7)

In [40]:
# id가 flaot으로 저장되어 있습니다. int로 바꿔주고 저장하겠습니다. 
idol_list['idol_id'] = idol_list['idol_id'].astype(int)
idol_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 7 columns):
artist         332 non-null object
artist_eng     331 non-null object
member_num     332 non-null object
leader_name    274 non-null object
gender         331 non-null object
agency         313 non-null object
idol_id        332 non-null int32
dtypes: int32(1), object(6)
memory usage: 16.9+ KB


In [41]:
# 수집을 위한 빈 리스트 생성

title = []
album = []
release_date = []
song_genre = []
like = []
lyrics = []
creator = []
artist = []
artist_m = []
debut_y = []
debut_t = []
agency = []
is_title = []
fan = []

In [42]:
# 곡마다 가수 이름을 붙여주어야 하기에 가수 이름 리스트로 받아오기
names = idol_list.artist
names[0]

'서태지와 아이들'

In [43]:
# 크롤러가 링크를 찾을 때 매번 바꿔주는 값. for문에 사용됩니다. 
ids = idol_list.idol_id
ids[0]

100161

In [44]:
# tests = ['780066']

## 크롤러

In [19]:
driver = webdriver.Chrome('C:/Users/pje17/Desktop/Lyrics/Crawlling/chromedriver')
driver.get("https://www.melon.com")

In [45]:
# 곡 정보를 찾아들어가기 위한 url을 분해해서 변수 지정해놓은 모습입니다. 
# l3 뒤에 붙는 숫자가 멜론 서버에 저장된 해당 가수의 해당 곡 번호입니다. 
# 레드벨벳의 32번째 곡은 뒤에 숫자가 32인 경우 곡 리스트에 첫번째로 보여집니다. 
# 즉 url을 바꿔가되 항상 리스트의 첫곡의 정보를 가져오면 모든 곡을 돌 수 있습니다. 

l1 = 'https://www.melon.com/artist/song.htm?artistId='
# idol_id
l2 = '#params%5BlistType%5D=A&params%5BorderBy%5D=ISSUE_DATE&params%5BartistId%5D='
# idol_id
l3 = '&po=pageObj&startIndex='
# song number
# l1 + 'idol_id' + l2 + 'idol_id' + l3 + number

# 가수 이름을 넣기 위한 변수
i = 0

for id in ids:
    
    n = 1
    while True:
                 
        #sleep(3)
        # 해당 가수의 곡 리스트 n번째로 가기
        
        driver.get(l1+str(id)+l2+str(id)+l3+str(n))    
        sleep(2)
        
        # 타이틀 여부 체크. 타이틀이 아니라면 NaN
        # 곡 세부 정보를 보기 전에 타이틀 태그를 확인해야함
        # 타이틀곡은 '타이틀 곡'이라고 수집이 됩니다. 
        try:
            is_title.append(driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr[1]/td[3]/div/div/span').text)
        except NoSuchElementException as e:
            is_title.append('')
                
        # 곡 정보로 들어가기
        try:
            driver.find_element_by_css_selector('tr:nth-child(1) > td:nth-child(3) > div > div > a.btn.btn_icon_detail > span').click()
            
            # 제목 크롤링(제목은 없지 않기 때문에 예외 없음)
            title.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]').text)

            # 앨범 크롤링
            try:
                album.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[1]/a').text)
            except NoSuchElementException as e:
                album.append('')

            # 발매날짜 크롤링
            try:
                release_date.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text)
            except NoSuchElementException as e:
                release_date.append('')

            try:
                song_genre.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text)
            except NoSuchElementException as e:
                song_genre.append('')

            # 좋아요 수 크롤링(좋아요도 0인 경우만 있고 없지는 않기에 예외 없음)
            like.append(driver.find_element_by_xpath('//*[@id="d_like_count"]').text)

            # 가사 크롤링
            try:
                lyrics.append(driver.find_element_by_xpath('//*[@id="d_video_summary"]').text)
            except NoSuchElementException as e:
                lyrics.append('')

            # 작사 작곡 내역이 없는 경우
            try:
                creator.append(driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul').text)
            except NoSuchElementException as e:
                creator.append('')

            n += 1
            
            # 가수 이름 붙이기
            artist.append(names[i])
            
            # 디버깅    
            print(len(title), title[len(title)-1])

        except NoSuchElementException as e:
            i += 1
            break
        
        

1 태지 보이스
2 시대유감(時代遺憾)
3 수시아(誰是我)
4 이 밤이 깊어 가지만 (Remix)
5 Free Style
6 난 알아요
7 너와 함께한 시간속에서
8 환상속의 그대
9 하여가
10 너에게
11 우리들만의 추억
12 Opening (Live)
13 마지막 축제 (Live)
14 발해를 꿈꾸며
15 영원
16 교실 이데아
17 널 지우려 해 (Live)
18 Come Back Home
19 슬픈 아픔
20 필승
21 Good Bye
22 이너비 리스너비 (Inst.)
23 Yo! Taiji
24 슬픈 아픔
25 필승(必勝)
26 Come Back Home
27 시대유감(詩代遺憾)
28 1996, 그들이 지구를 지배했을 때
29 Taiji Boys
30 Good Bye (Inst.)
31 Free Style
32 이너비리스너비 (Inst.)
33 Opening (Inst.)
34 널 지우려해
35 너에게
36 수시아
37 영원
38 Farewell To Love
39 내 맘이야
40 제킬 박사와 하이드
41 아이들의 눈으로
42 교실 이데아
43 발해를 꿈꾸며
44 상실
45 Yo! Taiji
46 아이들의 눈으로
47 내 맘이야
48 지킬박사와 하이드
49 영원(永遠)
50 발해를 꿈꾸며 (MR)
51 널 지우려 해
52 발해를 꿈꾸며
53 교실 이데아
54 Opening (The Taiji Boys)
55 마지막 축제
56 내 모든 것
57 환상속의 그대 (Part 5)
58 수시아
59 죽음의 늪
60 이밤이 깊어가지만
61 Taiji Solo
62 Juno Solo
63 Hyun-Seok Solo
64 너와 함께한 시간속에서
65 난 알아요
66 너에게
67 하여가
68 우리들만의 추억 Part 1
69 우리들만의 추억 Part 2
70 Yo! Taiji
71 하여가(何如歌)
72 우리들만의 추억
73 죽음의 늪
74 너에게
75 수시아(誰是我)
76 마지막 축제
77 우리들만의 추억
78 내 모든 것
79 환상속의 그대 Part Ⅱ
80 이

594 3!4!
595 오늘
596 그대 곁에서
597 우리에겐
598 흔들리며
599 아자
600 내 슬픈 사랑은
601 일,상,성
602 희망의 이름으로
603 다같이
604 너(조국)을 위한 시
605 Opening
606 꼬마 룰라
607 안녕
608 영웅은 어디로
609 프로와 아마추어
610 Hotel California
611 Korean In New York
612 마지막 사랑
613 사랑하기 때문에
614 후유증
615 내가 잠 못드는 이유
616 통일
617 친구를 보내며
618 날개 잃은 천사
619 Kulcha
620 비밀은 없어/100일째 만남
621 Roo`Ra In 1996
622 천상유애(天上有愛)
623 사랑법
624 하얀새
625 아스라이
626 인간인가?
627 착한 사랑
628 웃긴 아이
629 Rap-Hop
630 Come Together
631 천상유애(天上有愛) (Remix)
632 Roo`Ra In 1995
633 날개 잃은 천사
634 안녕
635 사랑의 기도
636 친구를 보내며
637 영웅은 어디로
638 프로와 아마츄어
639 그후로 3년후 (내가 잠못드는 이유 Ⅱ)
640 Hotel California
641 Koreana In New York
642 울면 안돼
643 루돌프 사슴코
644 Jingle Bell
645 석별의 정
646 Roots Of Reggae (Roo`Ra Thema)
647 100일(日)째 만남
648 너의 작은 기억 하나까지
649 공주와 하인
650 Roots Of Reggae
651 내가 잠못드는 이유
652 비밀은 없어
653 통일
654 100일(日)째 만남
655 100일(日)째 만남 (Inst.)
656 너의 작은 기억 하나까지 (Inst.)
657 White Christmas
658 날개잃은 천사
659 정리
660 Oh
661 It`s Mine
662 함께
663 후회
664 행복을 주는 사람
665 기대
666 Summer Of Love
667 희망의 이름으로
668 

1156 Look What You`ve Done (UK Acoustic Ver.)
1157 Look What You`ve Done
1158 Are You Gonna Be My Girl
1159 Are You Gonna Be My Girl
1160 Look What You`ve Done
1161 Look What You`ve Done
1162 Are You Gonna Be My Girl
1163 Cold Hard Bitch (Entourage 101)
1164 K.I.A. (Killed In Action)
1165 Beat On Repeat
1166 She`s A Genius
1167 Black Hearts (On Fire)
1168 Seventeen
1169 La Di Da
1170 Goodbye Hollywood
1171 Walk
1172 Times Like This
1173 Let Me Out
1174 Start The Show
1175 She Holds A Grudge
1176 She`s A Genius
1177 Rip It Up
1178 Bring It On Back (UK Acoustic Ver.)
1179 Kings Horses (UK Acoustic Ver.)
1180 Look What You`ve Done (UK Acoustic Ver.)
1181 Shine On (UK Acoustic Ver.)
1182 Are You Gonna Be My Girl (UK Acoustic Ver.)
1183 L`esprit D`escalier
1184 Holiday
1185 Put Your Money Where Your Mouth Is
1186 Bring It On Back
1187 That`s All Lies
1188 Kings Horses
1189 Shine On
1190 Come On Come On
1191 Stand Up
1192 Rip It Up
1193 Skin And Bones
1194 Shiny Magazine
1195 Eleanor
1196 Al

1524 상심(Ballad Ver.)
1525 수호천사(守護天使)
1526 후애(後愛)
1527 심연(深淵)
1528 Never Ending Story
1529 Forever With You
1530 이별공식
1531 그대만큼...
1532 찬란한 사랑
1533 부활을 기다리며
1534 귀머거리 하늘
1535 열병 48 °
1536 고요속의 외침
1537 하늘을 걸고
1538 발자욱
1539 Remember
1540 Jazz
1541 Break Beat
1542 Closing Ment
1543 Never Ending Story
1544 하늘을 걸고...
1545 그대만큼
1546 수호천사(守護天使)
1547 오랫동안...
1548 늘...
1549 행복한 슬픔
1550 우정의 이름으로
1551 부활을 기다리며...
1552 Outro
1553 귀머거리 하늘
1554 Freetro
1555 심연(深淵)
1556 마지막 잎새
1557 나의 형제
1558 아침에 떠나
1559 방황(彷湟) Ⅱ
1560 One Side...
1561 열병 48 °
1562 폭로
1563 후애
1564 Outro
1565 상심
1566 무(無)
1567 찬란한 사랑 (Classic Ver.)
1568 두번째 여행
1569 Good-Bye My Friend
1570 Rumors
1571 마음속을 걸어가 (Jungle Ver.)
1572 Jazz (New Jazz Ver.)
1573 가을의 기억 (Memories In Fall)
1574 서곡 (Back To The Black)
1575 마음속을 걸어가
1576 友情之思 (우정지사)
1577 니가 뭔데
1578 Jazz
1579 찬란한 사랑 (상심II)
1580 금지된 U-Turn
1581 방황 (彷徨)
1582 외로움을 기대며
1583 널 볼 수 없는 나는
1584 흔들리는 중심
1585 서곡 (Intro)
1586 이별 공식
1587 상심
1588 연인 아닌 사이
1589 슬픈 오해
1590 고요속의 외침
1591 엑스트라
1592 친구

1982 O.M.G (Oh My God)
1983 Sexy Music
1984 착각
1985 제8요일
1986 Intro (Nonstop Dance Ver.)
1987 겨울잠 자는 아이 (Nonstop Dance Ver.)
1988 난 봤어 (Nonstop Dance Ver.)
1989 본색 (Nonstop Dance Ver.)
1990 사자의 눈물 (Nonstop Dance Ver.)
1991 많이많이 (Nonstop Dance Ver.)
1992 안녕 (Nonstop Dance Ver.)
1993 게임의 법칙 (Nonstop Dance Ver.)
1994 열혈남아 (Nonstop Dance Ver.)
1995 비련 (Nonstop Dance Ver.)
1996 긴가민가 (Nonstop Dance Ver.)
1997 쇼크 (Nonstop Dance Ver.)
1998 Pandora (Nonstop Dance Ver.)
1999 천국보다 아름다운 (Nonstop Dance Ver.)
2000 아킬레스건 (Nonstop Dance Ver.)
2001 DNA (Nonstop Dance Ver.)
2002 Bye For You (Nonstop Dance Ver.)
2003 누가 그래 (Nonstop Dance Ver.)
2004 Target (Nonstop Dance Ver.)
2005 시련 (Nonstop Dance Ver.)
2006 위험한 초대 (Nonstop Dance Ver.)
2007 Sky Letter (Nonstop Dance Ver.)
2008 슬픈다짐 (Nonstop Dance Ver.)
2009 오늘도 행복해 (Nonstop Dance Ver.)
2010 Love Story (Nonstop Dance Ver.)
2011 다시 또 (Nonstop Dance Ver.)
2012 전봇대 (Nonstop Dance Ver.)
2013 Pain (Nonstop Dance Ver.)
2014 바다가 되어 (Nonstop Dance Ver.)
2015 너를 

...

16308 인터스텔라 (Feat. Yella Diamond) (Performed by 주헌, 형원, I.M)
16309 0 (YOUNG) (Feat. 노머시)
16310 Let Me Show You
16311 Let Me Show You (Inst.)
16312 HOW R U TODAY
16313 UP ALL NIGHT
16314 팔불출
16315 ANYWAY
16316 너 없는 난
16317 예쁘다 예뻐
16318 예쁘다 예뻐 (Inst.)
16319 그러니까 우리
16320 뜨거운 감자
16321 골목길에서
16322 I Know U Know
16323 이보다 좋을까
16324 딱 하루만
16325 Let`s Get Down To It
16326 진짜가 나타났다
16327 다행이야
16328 정리가 안돼
16329 짠해
16330 R U Ready?
16331 Lonely
16332 Knock Knock
16333 뻔뻔
16334 One N Only
16335 기가 막혀 (Awesome)
16336 가슴이 놀래 (Heartbreak)
16337 All In
16338 1분 (1 Minute)
16339 Endless Summer
16340 A-TEEN
16341 어쩌나
16342 Holiday
16343 나에게로 와
16344 What`s Good
16345 MOONWALKER
16346 우리의 새벽은 낮보다 뜨겁다
16347 Call Call Call!
16348 Highlight (Japanese Ver.)
16349 Lean on Me (Japanese Ver.)
16350 20 (Japanese Ver.)
16351 Love Letter (Japanese Ver.)
16352 단발머리
16353 Thinkin` about you
16354 고맙다
16355 지금 널 찾아가고 있어
16356 Falling For U
16357 Intro. 新世界
16358 CHANGE UP
16359 모자를 눌러 쓰고
16360 박수
16361 날 쏘고 가라
1636

16739 핑 (Crush)
16740 주머니 속 편지
16741 주머니 속 편지 (Inst.)
16742 주머니 속 편지 (Full Ver.)
16743 INTRO
16744 물들어요 (Fall in Love)
16745 너 그리고 나 (NAVILLERA)
16746 LOL
16747 한 뼘 (Distance)
16748 물꽃놀이 (Water Flower)
16749 Mermaid
16750 나의 일기장 (Sunshine)
16751 나침반 (Compas)
16752 찰칵 (Click)
16753 바람에 날려 (Gone with the wind)
16754 너 그리고 나 (NAVILLERA) (Inst.)
16755 INTRO (Snowflake)
16756 시간을 달려서 (Rough)
16757 내 이름을 불러줘 (Say my name)
16758 사랑별 (Luv Star)
16759 그런 날엔 (Someday)
16760 TRUST
16761 시간을 달려서 (Inst.)
16762 Intro (Flower Bud)
16763 오늘부터 우리는 (Me Gustas Tu)
16764 하늘 아래서
16765 One
16766 기억해 (My Buddy)
16767 오늘부터 우리는 (Me Gustas Tu) (Inst.)
16768 Intro (Season Of Glass)
16769 유리구슬 (Glass Bead)
16770 Neverland
16771 White (하얀마음)
16772 유리구슬 (Glass Bead) (Inst.)
16773 하얀 바람
16774 여우야
16775 터
16776 가시나무새
16777 혼미 (Feat. YunB)
16778 Beautiful Women
16779 Freedom
16780 Dream
16781 Love is
16782 Life (Feat. 매드클라운)
16783 Lonely Friday (Feat. 진실 Of Mad Soul Child)
16784 Upgrade
16785 Young & One
16786 BLACK D

17192 CHEER UP (Inst.) (CD ONLY)
17193 TT (Inst.) (CD ONLY)
17194 Y.M.C.A.
17195 I`m gonna be a star (CD Only)
17196 INTRO : Humming
17197 Moonlight
17198 Ride on the wind
17199 Knockin` on my heaven`s door
17200 Dimelo
17201 Ride on the wind (Inst.)
17202 INTO YOU
17203 Trust Me (J.seph&Jiwoo Ver.)
17204 Push & Pull
17205 지니까
17206 You In Me
17207 Trust Me (BM&Somin Ver.)
17208 이제 우리 (Song by 제이셉, 전소민)
17209 Oh NaNa (Hidden. 허영지)
17210 Don`t Recall
17211 RUMOR
17212 Hola Hola
17213 난 멈추지 않는다
17214 Living Good (Special thanks to.)
17215 RUMOR
17216 Don`t Recall
17217 Don`t Recall (Hidden Ver.)
17218 Oh NaNa (Hidden. 허영지)
17219 날개 잃은 천사
17220 INTO YOU (Inst.) (CD Only)
17221 You In Me (Inst.) (CD Only)
17222 Trust Me (CD Only)
17223 Oh NaNa (Inst.) (CD Only)
17224 Don`t Recall (Inst.) (CD Only)
17225 RUMOR (Inst.) (CD Only)
17226 Living Good (Inst.) (CD Only)
17227 너잖아 (Always you)
17228 너의 뒤에서 (By Your Side)
17229 외친다 (Call Out)
17230 내 곁에 있어줘 (Stay with me)
17231 Real Love
17232 그림자
1

17637 Shut Up (Feat. 유희열)
17638 왜 불러 (Feat. 미료)
17639 왜 불러 (Clean Ver.) (Feat. 미료)
17640 왜 불러 (Inst.)
17641 I.B.I
17642 몰래몰래
17643 NOT END...BUT AND !!
17644 Let`s Get Started
17645 The world is all one !!
17646 핑퐁게임 (PINGPONG GAME)
17647 I MUST GO!
17648 으르렁 (루키조)
17649 두번째 고백 (데뷔조)
17650 Wanna be your Star (하서&재인&예은)
17651 꿈을 Dream
17652 One For All
17653 메모리즈 (수지&지원)
17654 Lost In The Summer (재인&정주)
17655 꿈을 Dream
17656 One For All
17657 꿈을 Dream (Inst.)
17658 One For All (Inst.)
17659 집착
17660 Your Space
17661 입술에 입술 (Lip 2 Lip)
17662 쉿! (Shh!)
17663 MONSTER
17664 입술에 입술 (Lip 2 Lip) (Inst.)
17665 BAAM
17666 베리베리
17667 빙고게임
17668 Only one you
17669 BAAM (Inst.)
17670 안아줘
17671 안아줘
17672 안아줘 (Inst.)
17673 뿜뿜
17674 궁금해
17675 Same Same
17676 Fly
17677 뿜뿜 (Inst.)
17678 어마어마해 (EDM Ver.) (Inst.)
17679 꼼짝마
17680 좋아
17681 너, 어느 별에서 왔니
17682 오르골
17683 어마어마해 (EDM Ver.)
17684 꼼짝마 (Inst.)
17685 어마어마해 (EDM Ver.)
17686 어마어마해
17687 어마어마해 (Inst.)
17688 Welcome to MOMOLAND
17689 짠쿵쾅
17690 상사병
17691 

18069 너란 사람 (iTeen Girls Special)
18070 Neverland
18071 My World
18072 Fantastic
18073 Sweety
18074 ICE CHU
18075 ICE CHU (Inst.)
18076 느낌이 와
18077 아야
18078 나의 우주
18079 느낌이 와 (Inst.)
18080 Rolly
18081 ADD
18082 Sweet Crazy Love
18083 Uncover
18084 Girl Front
18085 LOONATIC
18086 Chaotic
18087 Starlight
18088 ODD Front
18089 ODD
18090 Girl Front
18091 LOONATIC
18092 Chaotic
18093 Starlight
18094 BLACK HEART
18095 비 내린 후에
18096 Moonlight
18097 BLACK HEART (Inst.)
18098 비 내린 후에 (Inst.)
18099 Moonlight (Inst.)
18100 그랬나봐
18101 감각
18102 ONLY ONE
18103 Ride with me
18104 믿어줘 (Rebooting)
18105 감각 (Inst.)
18106 ONLY ONE (Inst.)
18107 젊음의 노트
18108 TO.UNME (PRESENT) (CD Only)
18109 Dancing With The Devil (Live Ver.) (BLACK Ver. CD Only)
18110 끌어줘 (Live Ver.) (HEART Ver. CD Only)
18111 다가가고 싶어
18112 너를 따라, 너에게
18113 두근두근(DKDK)
18114 22세기 소녀
18115 CLOVER
18116 FIRST LOVE
18117 나에게로 오는 길 (Intro.)
18118 To Heart
18119 환상속의 그대
18120 피노키오
18121 Be With You
18122 유리구두 (MAMA Ver.)
18123 유리구두
18124 バナナが食

## Dataframe에 크롤링된 데이터 저장하기

In [46]:
test_list = [title, album, release_date, song_genre, is_title, like, creator, lyrics]
column_names = ['title', 'artist', 'album', 'release_date', 'song_genre', 'is_title', 'like', 'creator', 'lyrics']
column_v = [title, artist, album, release_date, song_genre, is_title, like, creator, lyrics]

test_df = pd.DataFrame(column_v)

# 이렇게 집어넣었더니 행이 9개인 데이터프레임이 나와버립니다. 행렬을 바꾸어줍니다. 

In [47]:
test_p = test_df.head(9).T
test_p.head()

,0,1,2,3,4,5,6,7,8
0,태지 보이스,서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,,269,서태지\n작사\n서태지\n작곡\n서태지\n편곡,갈수 없는 곳이란 우리에겐 없어\n보이는 길밖에도 세상은 있어\n언제나 식지않는 마...
1,시대유감(時代遺憾),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,타이틀 곡,"3,265",서태지\n작사\n서태지\n작곡\n서태지\n편곡,왜 기다려 왔잖아\n모든 삶을 포기하는 소리를\n이 세상이 모두 미쳐버릴\n일이 벌...
2,수시아(誰是我),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,,488,서태지\n작사\n서태지\n작곡\n서태지\n편곡,이 세상 그 누구도 나를 닮을 순 없네\n날 세상에 알릴 거야\n나 역시 그 누구를...
3,이 밤이 깊어 가지만 (Remix),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,,702,양현석\n작사\n서태지\n작곡\n서태지\n편곡,옛 생각에 까페 문을열고\n지난 추억을 기억하려했지\n부드러운 음악소리마저\n내마음...
4,Free Style,서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,,602,서태지\n작사\n김종서\n작사\n서태지\n작곡\n김종서\n작곡\n서태지\n편곡,항상 난 지나치기만 했네\n이젠 난 미련없이 끝내\n됐어 찾아냈어 난\n모두가 또 ...


In [49]:
test_p.columns = column_names
test_p.tail(350)

,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics
18165,3rd Eye,스트레이 키즈,I am NOT,2018.03.26,Rap / Hip-hop,타이틀 곡,"2,176",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,눈을 감아 보이는 3rd Eye\n원하는 걸 보여줘 3rd Eye\nI just w...
18166,Hellevator,스트레이 키즈,Mixtape,2018.01.08,Electronica,타이틀 곡,"7,418",아르마딜로\n작사\n방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한...,내 삶은 밑바닥 낭떠러지\n어두운 터널 속을 걷고 있어\n내게 잔인하고 두려운 하루...
18167,Grrr 총량의 법칙,스트레이 키즈,Mixtape,2018.01.08,Rap / Hip-hop,,"3,650",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,총량의 법칙\nGrrr 정신은 지저분해\n난폭한 개마냥 누구 앞에\n서 있든지 상관...
18168,어린 날개,스트레이 키즈,Mixtape,2018.01.08,Rap / Hip-hop,,"3,938",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,어리게 보이기 싫어서\n괜히 짓궂은 옷만 계속 바꿔 입고\n어른스럽고만 싶어서\n괜...
18169,YAYAYA,스트레이 키즈,Mixtape,2018.01.08,Rap / Hip-hop,,"2,893",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,Stray Kids\nWatch out\n2018\n야야 야야 야야 야야 야야 야야...
18170,GLOW,스트레이 키즈,Mixtape,2018.01.08,Ballad,,"2,853",창빈 (3RACHA)\n작사\n필릭스\n작사\n리노\n작사\n방찬 (3RACHA)\...,쉬어도 되지만 아직 일러\n쉬어야 하지만 아직 멀어\n우리가 가야 할 길\n끝이 안...
18171,School Life,스트레이 키즈,Mixtape,2018.01.08,Dance,,"2,685",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,아침 7시 알람이 댕\n창문 사이 햇빛은 쨍\n일어나란 엄마의 잔소리는\n마치 한여...
18172,4419,스트레이 키즈,Mixtape,2018.01.08,Rap / Hip-hop,인기 곡,"2,935",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,너는 어떻게 지내\n텅 빈 내 옆자리는 허전해\n잊을 수 없는 날들 뒤로해도\n그저...
18173,Hellevator,스트레이 키즈,Hellevator,2017.11.01,Dance,,"7,739",아르마딜로\n작사\n방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한...,내 삶은 밑바닥 낭떠러지\n어두운 터널 속을 걷고 있어\n내게 잔인하고 두려운 하루...
18174,Mixtape#2,스트레이 키즈,I am WHO,2018.08.06,Rap / Hip-hop,타이틀 곡,270,방찬 (3RACHA)\n작사\n우진\n작사\n리노\n작사\n창빈 (3RACHA)\n...,


In [50]:
test_p.shape

(18515, 9)

In [51]:
# 간혹 이상하게 수집된 곡들이 많습니다. 이런 곡들은 전처리에서 해결해줍시다.
test_p['title'].isnull().sum()

332

In [52]:
test_p.tail()

,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics
18510,None,None,None,None,None,타이틀 곡,None,None,None
18511,None,None,None,None,None,,None,None,None
18512,None,None,None,None,None,,None,None,None
18513,None,None,None,None,None,,None,None,None
18514,None,None,None,None,None,,None,None,None


In [53]:
test_p.to_csv('song_data_raw.csv')